In [94]:
import pandas as pd

melb_data = pd.read_csv('data/melb_data_ps.csv', sep=',')
melb_df = melb_data.copy()
melb_df=melb_data.drop(['index', 'Coordinates'], axis=1)
total_rooms = melb_df['Rooms'] + melb_df['Bedroom'] + melb_df['Bathroom']
melb_df['MeanRoomsSquare'] = melb_df['BuildingArea'] / total_rooms
diff_area = melb_df['BuildingArea'] - melb_df['Landsize']
sum_area = melb_df['BuildingArea'] + melb_df['Landsize']
melb_df['AreaRatio'] = diff_area/sum_area
melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
melb_df['MonthSale'] = melb_df['Date'].dt.month
delta_days = melb_df['Date'] - pd.to_datetime('2016-01-01') 
melb_df['AgeBuilding'] = melb_df['Date'].dt.year - melb_df['YearBuilt']
melb_df = melb_df.drop('YearBuilt', axis=1)
melb_df['WeekdaySale']=melb_df['Date'].dt.dayofweek
mask1=(melb_df['WeekdaySale']==6) | (melb_df['WeekdaySale']==5)
#display(melb_df[mask1].shape[0])
#display(melb_df['Address'].nunique())
def get_street_type(address):
    exclude_list = ['N', 'S', 'W', 'E']
    address_list = address.split(' ')
    street_type = address_list[-1]
    if street_type in exclude_list:
        street_type = address_list[-2]
    return street_type
street_types = melb_df['Address'].apply(get_street_type)
popular_stypes =street_types.value_counts().nlargest(10).index
melb_df['StreetType'] = street_types.apply(lambda x: x if x in popular_stypes else 'other')
melb_df = melb_df.drop('Address', axis=1)

def get_weekend(weekday):
    if weekday in (5,6): return 1
    else: return 0
melb_df['Weekend'] = melb_df['WeekdaySale'].apply(get_weekend)
mask2=melb_df['Weekend'] == 1
popular_sell=melb_df['SellerG'].value_counts().nlargest(49).index
melb_df['SellerG'] = melb_df['SellerG'].apply(lambda x: x if x in popular_sell else 'other')
mask3=melb_df['SellerG']=='Nelson'; mask4=melb_df['SellerG']=='other'
#display(melb_df[mask3]['Price'].min()/melb_df[mask4]['Price'].min())
unique_list = []
for col in melb_df.columns:
    item = (col, melb_df[col].nunique(),melb_df[col].dtypes) 
    unique_list.append(item) 
unique_counts = pd.DataFrame(
    unique_list,
    columns=['Column_Name', 'Num_Unique', 'Type']
).sort_values(by='Num_Unique',  ignore_index=True)
#display(unique_counts)
cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] # список столбцов, которые мы не берём во внимание
max_unique_count = 150 # задаём максимальное число уникальных категорий
for col in melb_df.columns: # цикл по именам столбцов
    if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude: # проверяем условие
        melb_df[col] = melb_df[col].astype('category') # преобразуем тип столбца
#display(melb_df['Regionname'].cat.categories)
#display(melb_df['Regionname'].cat.codes)
melb_df['Type'] = melb_df['Type'].cat.rename_categories({
    'u': 'unit',
    't': 'townhouse',
    'h': 'house'
})
#display(melb_df['Type'])
new_houses_types = pd.Series(['unit', 'house', 'flat', 'flat', 'house'])
new_houses_types = new_houses_types.astype(melb_df['Type'].dtype)
#display(new_houses_types)
melb_df['Type'] = melb_df['Type'].cat.add_categories('flat')
new_houses_types = pd.Series(['unit', 'house', 'flat', 'flat', 'house'])
new_houses_types = new_houses_types.astype(melb_df['Type'].dtype)
popular_sub = melb_df['Suburb'].value_counts().nlargest(119)
melb_df['Suburb'] = melb_df['Suburb'].apply(lambda x: x if x in popular_sub else 'other')
melb_df['Suburb'] = melb_df['Suburb'].astype('category') # преобразуем тип столбца
display(melb_df['Suburb'].cat.codes)
#display(melb_df['Suburb'].cat.categories)

0          0
1          0
2          0
3          0
4          0
        ... 
13575    119
13576    116
13577    116
13578    116
13579    118
Length: 13580, dtype: int8

In [95]:
import pandas as pd
rough_data= pd.read_csv('https://raw.githubusercontent.com/justmarkham/pandas-videos/master/data/ufo.csv', sep=',')
UFO_data=rough_data.copy()
UFO_data['Time']=pd.to_datetime(UFO_data['Time'], dayfirst=True)
UFO_data['Year']=UFO_data['Time'].dt.year
UFO_data['Month']=UFO_data['Time'].dt.month
UFO_data['Day']=UFO_data['Time'].dt.day
mask=UFO_data['State']=='NV'

display(UFO_data[mask]['Time'].diff().mean())

Timedelta('68 days 17:19:36.678445230')

In [190]:
rough_data = pd.read_csv('data/citibike-tripdata.csv', sep=',')
velo_df = rough_data.copy()
velo_df=velo_df.drop(['start station id', 'end station id'], axis=1)
#display(velo_df['start station name'].nunique(), velo_df['end station name'].nunique())
velo_df['age']=2018-velo_df['birth year']
velo_df=velo_df.drop(['birth year'], axis=1)
#display(velo_df[velo_df['age']>60]['age'].count())
velo_df['stoptime']=pd.to_datetime(velo_df['stoptime'])
velo_df['starttime']=pd.to_datetime(velo_df['starttime'])
velo_df['trip duration']=velo_df['stoptime']-velo_df['starttime']
week_ends=pd.to_datetime(['2018-09-01','2018-09-02'])
def day_check(date):
    if date in week_ends:return 1
    else: return 0
velo_df['date']=pd.to_datetime(velo_df['starttime'].dt.date)
velo_df['weekend']=velo_df['date'].apply(day_check)
def day_div(quoter):
    if 0<=quoter<=6 : return 'night'
    elif 6<quoter<=12 : return 'morning'
    elif 12<quoter<=18 : return 'day'
    else: return 'evening'
    
velo_df['time_of_day']=velo_df['starttime'].dt.hour
velo_df['time_of_day']=velo_df['time_of_day'].apply(day_div)
display(velo_df['time_of_day'].value_counts())

day        143012
morning     95530
evening     46373
night       15085
Name: time_of_day, dtype: int64